In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
dvfs_list = ['1', '0c00', '0d00', '0e00', 
             '0f00', '1000', '1100', '1200', 
             '1300', '1400', '1500', '1600', '1700', '1800', '1900', '1a00']
dvfs_dict = {
    "0x1" : 1,
    "0x0c00" :  1.2,
    "0x0d00" :  1.3,
    "0x0e00" :  1.4,
    "0x0f00" :  1.5,
    "0x1000" : 1.6,
    "0x1100" : 1.7,
    "0x1200" : 1.8,
    "0x1300" : 1.9,
    "0x1400" : 2.0,
    "0x1500" : 2.1,
    "0x1600" : 2.2,
    "0x1700" : 2.3,
    "0x1800" : 2.4,
    "0x1900" : 2.5,
    "0x1a00" : 2.6,
    "0x1b00" : 2.7,
    "0x1c00" : 2.8,
    "0x1d00" : 2.9,
}

df_dict = {
    'i': [], 'itr': [], 'dvfs': [], 'rate': [], 'policy': [], 'watts_avg': [], 'watts_std': [], 
    'source': [], 'mapper':[], 'sink':[],
    
    'pollCnt': [], 'c1Cnt': [], 'c1eCnt': [],'c3Cnt': [], 'c6Cnt': [], 
    'rxPackets': [], 'rxBytes': [], 'txPackets': [], 'txBytes': [],
    'erxPackets': [], 'erxBytes':[], 'etxPackets': [], 'etxBytes':[],
    
    'SinknumRecordsInPerSecond_avg': [], 'SinknumRecordsInPerSecond_std': [], 
    'SinknumRecordsOutPerSecond_avg': [], 'SinknumRecordsOutPerSecond_std': [], 
    'SinkbusyTimeMsPerSecond_avg': [], 'SinkbusyTimeMsPerSecond_std': [], 
    'SinkbackPressuredTimeMsPerSecond_avg': [], 'SinkbackPressuredTimeMsPerSecond_std': [], 
    'SinkbusyTime_%': [], 'SinkbackPressuredTime_%': [], 

    'SourcenumRecordsInPerSecond_avg': [], 'SourcenumRecordsInPerSecond_std': [], 
    'SourcenumRecordsOutPerSecond_avg': [], 'SourcenumRecordsOutPerSecond_std': [], 
    'SourcebusyTimeMsPerSecond_avg': [], 'SourcebusyTimeMsPerSecond_std': [], 
    'SourcebackPressuredTimeMsPerSecond_avg': [], 'SourcebackPressuredTimeMsPerSecond_std': [], 
    'SourcebusyTime_%': [], 'SourcebackPressuredTime_%': [], 

    'MappernumRecordsInPerSecond_avg': [], 
    'MappernumRecordsInPerSecond_std': [], 'MappernumRecordsOutPerSecond_avg': [], 
    'MappernumRecordsOutPerSecond_std': [], 'MapperbusyTimeMsPerSecond_avg': [], 
    'MapperbusyTimeMsPerSecond_std': [], 'MapperbackPressuredTimeMsPerSecond_avg': [], 
    'MapperbackPressuredTimeMsPerSecond_std': [],
    'MapperbusyTime_%': [], 'MapperbackPressuredTime_%': []
}

print(df_dict)
print("*****************************************************************")

def parseFile(loc, rate, itr, dvfs, policy, i, nsrc, nmapper, nsink):
    file=f"{loc}/query1_cores16_frate{rate}_600000_fbuff-1_itr{itr}_{policy}dvfs{dvfs}_source{nsrc}_mapper{nmapper}_sink{nsink}_repeat{i}/summary.csv"
    print(file)
    df_dict['i'].append(i)
    df_dict['itr'].append(itr)
    if '0x'+str(dvfs) in dvfs_dict:
        df_dict['dvfs'].append(dvfs_dict['0x'+str(dvfs)])
    else:
        df_dict['dvfs'].append(dvfs)
    df_dict['policy'].append(policy)
    df_dict['rate'].append(rate)
    df_dict['source'].append(nsrc)
    df_dict['mapper'].append(nmapper)
    df_dict['sink'].append(nsink)
    
    df = pd.read_csv(file)
    df = df[df.columns.drop(list(df.filter(regex='Cnt')))]
    df = df[df.columns.drop(list(df.filter(regex='Bytes')))]
    #print(df)
    
    dff = df[df['name'].str.contains('Sink')]
    dff.columns = 'Sink' + dff.columns
    cols = dff.columns
    for col in cols[2:]:
        df_dict[col].append(dff.mean(numeric_only=True)[col])

    dff = df[df['name'].str.contains('Source')]
    dff.columns = 'Source' + dff.columns
    cols = dff.columns
    for col in cols[2:]:
        df_dict[col].append(dff.mean(numeric_only=True)[col])

    dff = df[df['name'].str.contains('Mapper')]
    dff.columns = 'Mapper' + dff.columns
    cols = dff.columns
    for col in cols[2:]:
        df_dict[col].append(dff.mean(numeric_only=True)[col])

    jfile = f"{loc}/query1_cores16_frate{rate}_600000_fbuff-1_itr{itr}_{policy}dvfs{dvfs}_source{nsrc}_mapper{nmapper}_sink{nsink}_repeat{i}/server2_rapl.log"
    with open(jfile) as file:
        lines = [float(line.rstrip()) for line in file]
        print(len(lines))
        df_dict['watts_avg'].append(float(round(np.mean(lines[300:600]), 2)))
        df_dict['watts_std'].append(float(round(np.std(lines[300:600]), 2)))
        #print(np.std(lines[300:500]))
        
    jfile = f"{loc}/query1_cores16_frate{rate}_600000_fbuff-1_itr{itr}_{policy}dvfs{dvfs}_source{nsrc}_mapper{nmapper}_sink{nsink}_repeat{i}/stats.csv"
    with open(jfile) as file:
        poll = []
        c1 = []
        c1e = []
        c3 = []
        c6 = []
        rxp = []
        rxb = []
        txp = []
        txb = []
        erxp = []
        erxb = []
        etxp = []
        etxb = []
        for line in file:
            ll = [int(a) for a in line.strip().split(',')]
            poll.append(ll[0])
            c1.append(ll[1])
            c1e.append(ll[2])
            c3.append(ll[3])
            c6.append(ll[4])
            rxp.append(ll[5])
            rxb.append(ll[6])
            txp.append(ll[7])
            txb.append(ll[8])
            erxp.append(ll[9])
            erxb.append(ll[10])
            etxp.append(ll[11])
            etxb.append(ll[12])
        ss = 30
        ee = 60
        df_dict['pollCnt'].append(np.sum(poll[ss:ee]))
        df_dict['c1Cnt'].append(np.sum(c1[ss:ee]))
        df_dict['c1eCnt'].append(np.sum(c1e[ss:ee]))
        df_dict['c3Cnt'].append(np.sum(c3[ss:ee]))
        df_dict['c6Cnt'].append(np.sum(c6[ss:ee]))
        #'rxPackets': [], 'rxBytes': [], 'txPackets': [], 'txBytes': [],
        df_dict['rxPackets'].append(np.sum(rxp[ss:ee]))
        df_dict['rxBytes'].append(np.sum(rxb[ss:ee]))        
        df_dict['txPackets'].append(np.sum(txp[ss:ee]))
        df_dict['txBytes'].append(np.sum(txb[ss:ee]))
        df_dict['erxPackets'].append(np.sum(erxp[ss:ee]))
        df_dict['erxBytes'].append(np.sum(erxb[ss:ee]))
        #print(np.mean(erxb[30:50]), np.std(erxb[30:50]))
        df_dict['etxPackets'].append(np.sum(etxb[ss:ee]))
        df_dict['etxBytes'].append(np.sum(etxb[ss:ee]))

{'i': [], 'itr': [], 'dvfs': [], 'rate': [], 'policy': [], 'watts_avg': [], 'watts_std': [], 'source': [], 'mapper': [], 'sink': [], 'pollCnt': [], 'c1Cnt': [], 'c1eCnt': [], 'c3Cnt': [], 'c6Cnt': [], 'rxPackets': [], 'rxBytes': [], 'txPackets': [], 'txBytes': [], 'erxPackets': [], 'erxBytes': [], 'etxPackets': [], 'etxBytes': [], 'SinknumRecordsInPerSecond_avg': [], 'SinknumRecordsInPerSecond_std': [], 'SinknumRecordsOutPerSecond_avg': [], 'SinknumRecordsOutPerSecond_std': [], 'SinkbusyTimeMsPerSecond_avg': [], 'SinkbusyTimeMsPerSecond_std': [], 'SinkbackPressuredTimeMsPerSecond_avg': [], 'SinkbackPressuredTimeMsPerSecond_std': [], 'SinkbusyTime_%': [], 'SinkbackPressuredTime_%': [], 'SourcenumRecordsInPerSecond_avg': [], 'SourcenumRecordsInPerSecond_std': [], 'SourcenumRecordsOutPerSecond_avg': [], 'SourcenumRecordsOutPerSecond_std': [], 'SourcebusyTimeMsPerSecond_avg': [], 'SourcebusyTimeMsPerSecond_std': [], 'SourcebackPressuredTimeMsPerSecond_avg': [], 'SourcebackPressuredTimeMsPe

In [5]:
for qps in [100000, 200000, 300000]:
    for tmp in [2, 4, 8]:
        parseFile("/home/han/github/logs/threeNodes_10_23_2023", qps, 1, 1, "ondemand", 0, tmp, 16, 16)

/home/han/github/logs/threeNodes_10_23_2023/query1_cores16_frate100000_600000_fbuff-1_itr1_ondemanddvfs1_source2_mapper16_sink16_repeat0/summary.csv
698
/home/han/github/logs/threeNodes_10_23_2023/query1_cores16_frate100000_600000_fbuff-1_itr1_ondemanddvfs1_source4_mapper16_sink16_repeat0/summary.csv
703
/home/han/github/logs/threeNodes_10_23_2023/query1_cores16_frate100000_600000_fbuff-1_itr1_ondemanddvfs1_source8_mapper16_sink16_repeat0/summary.csv
699
/home/han/github/logs/threeNodes_10_23_2023/query1_cores16_frate200000_600000_fbuff-1_itr1_ondemanddvfs1_source2_mapper16_sink16_repeat0/summary.csv
698
/home/han/github/logs/threeNodes_10_23_2023/query1_cores16_frate200000_600000_fbuff-1_itr1_ondemanddvfs1_source4_mapper16_sink16_repeat0/summary.csv
700
/home/han/github/logs/threeNodes_10_23_2023/query1_cores16_frate200000_600000_fbuff-1_itr1_ondemanddvfs1_source8_mapper16_sink16_repeat0/summary.csv
701
/home/han/github/logs/threeNodes_10_23_2023/query1_cores16_frate300000_600000_fbuf

In [6]:
#parseFile("/home/han/github/logs/threeNodes_10_23_2023", qps, 1, 1, "ondemand", 0, 16, 16, tmp)

d = pd.DataFrame(df_dict)
print(d.columns)

Index(['i', 'itr', 'dvfs', 'rate', 'policy', 'watts_avg', 'watts_std',
       'source', 'mapper', 'sink', 'pollCnt', 'c1Cnt', 'c1eCnt', 'c3Cnt',
       'c6Cnt', 'rxPackets', 'rxBytes', 'txPackets', 'txBytes', 'erxPackets',
       'erxBytes', 'etxPackets', 'etxBytes', 'SinknumRecordsInPerSecond_avg',
       'SinknumRecordsInPerSecond_std', 'SinknumRecordsOutPerSecond_avg',
       'SinknumRecordsOutPerSecond_std', 'SinkbusyTimeMsPerSecond_avg',
       'SinkbusyTimeMsPerSecond_std', 'SinkbackPressuredTimeMsPerSecond_avg',
       'SinkbackPressuredTimeMsPerSecond_std', 'SinkbusyTime_%',
       'SinkbackPressuredTime_%', 'SourcenumRecordsInPerSecond_avg',
       'SourcenumRecordsInPerSecond_std', 'SourcenumRecordsOutPerSecond_avg',
       'SourcenumRecordsOutPerSecond_std', 'SourcebusyTimeMsPerSecond_avg',
       'SourcebusyTimeMsPerSecond_std',
       'SourcebackPressuredTimeMsPerSecond_avg',
       'SourcebackPressuredTimeMsPerSecond_std', 'SourcebusyTime_%',
       'SourcebackPressuredTi

In [7]:
d[(d.rate == 100000)][['rate', 'policy', 'source', 'mapper', 'sink', 'SourcenumRecordsOutPerSecond_avg', 'SinkbusyTime_%', 'SinkbackPressuredTime_%', 'MapperbackPressuredTime_%']]

,rate,policy,source,mapper,sink,SourcenumRecordsOutPerSecond_avg,SinkbusyTime_%,SinkbackPressuredTime_%,MapperbackPressuredTime_%
0,100000,ondemand,16,16,2,99982.913229,12.390000,0.0,0.000000
1,100000,ondemand,16,16,4,99971.637535,17.526667,0.0,0.000000
2,100000,ondemand,16,16,8,99994.178785,10.906250,0.0,0.000000
3,100000,ondemand,16,16,16,100000.000000,5.956250,0.0,0.002917
12,100000,ondemand,2,16,16,100000.000000,1.073125,0.0,0.000000
13,100000,ondemand,4,16,16,100000.000000,0.910208,0.0,0.000000
14,100000,ondemand,8,16,16,100000.000000,1.522500,0.0,0.000000


In [8]:
d[(d.rate == 200000)][['rate', 'policy', 'source', 'mapper', 'sink', 'SourcenumRecordsOutPerSecond_avg', 'SinkbusyTime_%', 'SinkbackPressuredTime_%', 'MapperbackPressuredTime_%']]

,rate,policy,source,mapper,sink,SourcenumRecordsOutPerSecond_avg,SinkbusyTime_%,SinkbackPressuredTime_%,MapperbackPressuredTime_%
4,200000,ondemand,16,16,2,199983.883627,53.603333,0.0,2.435625
5,200000,ondemand,16,16,4,200000.000000,40.090833,0.0,0.000417
6,200000,ondemand,16,16,8,200000.000000,21.250000,0.0,0.001042
7,200000,ondemand,16,16,16,199980.763877,11.127708,0.0,0.007500
15,200000,ondemand,2,16,16,199993.776667,2.070833,0.0,0.000000
16,200000,ondemand,4,16,16,200000.000000,1.503542,0.0,0.000000
17,200000,ondemand,8,16,16,200000.000000,6.023958,0.0,0.000000


In [9]:
d[(d.rate == 300000)][['rate', 'policy', 'source', 'mapper', 'sink', 'SourcenumRecordsOutPerSecond_avg', 'SinkbusyTime_%', 'SinkbackPressuredTime_%', 'MapperbackPressuredTime_%']]

,rate,policy,source,mapper,sink,SourcenumRecordsOutPerSecond_avg,SinkbusyTime_%,SinkbackPressuredTime_%,MapperbackPressuredTime_%
8,300000,ondemand,16,16,2,299986.639401,84.826667,0.0,11.783750
9,300000,ondemand,16,16,4,299976.818914,46.878333,0.0,0.000000
10,300000,ondemand,16,16,8,300005.496076,32.177917,0.0,0.002292
11,300000,ondemand,16,16,16,299994.939757,16.779167,0.0,0.001875
18,300000,ondemand,2,16,16,299974.655000,3.069792,0.0,0.000000
19,300000,ondemand,4,16,16,299964.738472,2.472500,0.0,0.000000
20,300000,ondemand,8,16,16,300000.000000,9.630417,0.0,0.000000
